In [88]:
import LCCall as LCC
import LCAnalyses as LCA
import LCPlot as LCP
import importlib
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import patches
importlib.reload(LCC)
importlib.reload(LCA)
importlib.reload(LCP)

<module 'LCPlot' from '/mnt/Secdrive/TSmain/Pip_test/trigger_codes_github/LCPlot.py'>

In [89]:
%matplotlib tk
k = "EC53"
mod_date = "191021"
wl = "UKIRT_K"
region = "SERPM"
input_dir = '/mnt/Secdrive/TSmain/SERPM/EC53/EC53/'
output_dir = "/mnt/Secdrive/TSmain/SERPM/EC53/InCanada/"
dates_K, mags_K, noises_K = LCC.EC53IR("UKIRT_K",imeunit="day",addLiverpool=0,timeunit="day")
dates_H, mags_H, noises_H = LCC.EC53IR("UKIRT_H",imeunit="day",addLiverpool=0,timeunit="day")
dates_J, mags_J, noises_J = LCC.EC53IR("UKIRT_J",imeunit="day",addLiverpool=0,timeunit="day")
dates_LH, mags_LH, noises_LH = LCC.EC53IR("Liverpool_H",imeunit="day",timeunit="day")
JDs_W1, mags_W1, noises_W1 = LCC.EC53IR("WISE_W1_avg")
JDs_W2, mags_W2, noises_W2 = LCC.EC53IR("WISE_W2_avg")

JDs,Js,Std_Js,Hs,Std_Hs,Ks,Std_Ks,J_Hs,J_H_errs,H_Ks,H_K_errs, = np.loadtxt(input_dir +"/EC53_UKIRT_JHK_combined_fin.dat", unpack=True, dtype='float')
dates_850, fluxes_850, noises_850 = LCC.JCMTTRansientSource(k,"SERPM","850",'0410',test=1)
dates_850 = dates_850[:-1]
fluxes_850 = fluxes_850[:-1]
noises_850 = [np.sqrt((x**2+0.02*0.02*np.mean(fluxes_850)**2)) for x in noises_850[:-1]]
lc = LCC.EC53call()


In [90]:
plt.close()
plt.scatter(JDs,Js)
plt.gca().invert_yaxis()

In [179]:
JDs_W1 += 2400000.5
JDs_W2 += 2400000.5

In [37]:
crosspoints = [(2457161.555120096, 14.6161034454326),
 (2457211.107483692, 13.002953466189865),
 (2457620.4539564606, 14.65195122274963),
 (2457704.142392756, 11.927520146693496),
 (2458202.47722917, 13.934995676418112),
 (2458263.0412291205, 11.963367924010527),
 (2458752.47722917, 13.93499567641993)]
crossx = np.array([x[0] for x in crosspoints])
crossy = np.array([x[1] for x in crosspoints])

In [91]:
importlib.reload(LCA)
offset_factors1 = [1.0, 0.95, 0.79, 0.79]
offset_factors = [1.01, 0.96, 0.78, 0.74]
offset_factors = [1.01, 0.96, 0.78, 0.75]
offset2_factors = [0.6,0.55,0.44,0.24]
per = 530
phoff = 200 #former 170
subphoff=phoff
#subphoff = (dates_850[0]-JDs[0]+phoff)%525/525*565

check_phup = LCA.LCphaseup(crossx,crossy,"UKIRT_H",offset=offset_factors[1],offset2=offset2_factors[1],period=per,offset_date=phoff)

J_phup = LCA.LCphaseup(JDs,Js,"UKIRT_J",offset=offset_factors[0],offset2=offset2_factors[0],period=per,offset_date=phoff)
H_phup = LCA.LCphaseup(JDs,Hs,"UKIRT_H",offset=offset_factors[1],offset2=offset2_factors[1],period=per,offset_date=phoff)
K_phup = LCA.LCphaseup(JDs,Ks,"UKIRT_K",offset=offset_factors[2],offset2=offset2_factors[2],period=per,offset_date=phoff)
Sub_phup = LCA.LCphaseup(np.array(dates_850),np.array(fluxes_850),"SCUBA2_850",offset=offset_factors[3],offset2=offset2_factors[3],offset_date=subphoff,period=per)
W1_phup = LCA.LCphaseup(JDs_W1,mags_W1,"WISE_W1",offset=0.8,period=per,offset_date=phoff)
JH_phup = LCA.LCphaseup(JDs,J_Hs,'UKIRT_JH',offset=offset_factors[0]-offset_factors[1],offset2=offset2_factors[0]-offset2_factors[1],period=per,offset_date=phoff)
HK_phup = LCA.LCphaseup(JDs,H_Ks,'UKIRT_HK',offset=offset_factors[1]-offset_factors[2],offset2=offset2_factors[1]-offset2_factors[2],period=per,offset_date=phoff)

phases = [0.4,0.7]
burstlabel = '{0:0.1f}'.format(phases[0])+" < Phase < " + '{0:0.1f}'.format(phases[1])
HK_burstx,HK_burst = HK_phup.Phasepickup(phases)
JH_burstx,JH_burst = JH_phup.Phasepickup(phases)
H_burstx,H_burst = H_phup.Phasepickup(phases)
J_burstx,J_burst = J_phup.Phasepickup(phases)
K_burstx, K_burst = K_phup.Phasepickup(phases)
Sub_burstx,Sub_burst = Sub_phup.Phasepickup(phases)
spliter = [0.1,0.3,0.5,0.7,0.9]
J_splitx,J_split = J_phup.Phasesplit(spliter)
JH_splitx,JH_split = JH_phup.Phasesplit(spliter)
H_splitx,H_split = H_phup.Phasesplit(spliter)
HK_splitx,HK_split = HK_phup.Phasesplit(spliter)
J_phup.Phasextend()
H_phup.Phasextend()
K_phup.Phasextend()
Sub_phup.Phasextend()
W1_phup.Phasextend()
JH_phup.Phasextend()
HK_phup.Phasextend()
check_phup.Phasextend()

Jcut= J_phup.cut_ind
Jcut2 = J_phup.cut_ind2
Hcut= H_phup.cut_ind
Hcut2= H_phup.cut_ind2
Kcut= K_phup.cut_ind
Kcut2= K_phup.cut_ind2
Subcut= Sub_phup.cut_ind
Subcut2= Sub_phup.cut_ind2
W1cut= W1_phup.cut_ind


In [93]:
save =0

In [ ]:
asdf = 0
fdsa = 0
for i in [0,1,2,3]:
    asdf += np.size(JH_split[i])
    fdsa += np.size(J_split[i])
    


In [25]:
len(Sub_phup.xplot)

51

In [75]:
save =1
exterr = list(lc.mnoises_850[:-1]) + list(lc.mnoises_850[:-1]) + list(lc.mnoises_850[:-1])

In [59]:
for i in range(0,21,7):
    print(check_phup.yplott[i:i+7])

0
[13.6561034454326, 12.042953466189864, 13.69195122274963, 11.927520146693496, 13.934995676418112, 11.963367924010527, 13.93499567641993]
7
[13.6561034454326, 12.042953466189864, 13.69195122274963, 11.927520146693496, 13.934995676418112, 11.963367924010527, 13.93499567641993]
14
[13.6561034454326, 12.042953466189864, 13.69195122274963, 11.927520146693496, 13.934995676418112, 11.963367924010527, 13.93499567641993]


In [92]:
plt.close()
fig, ax = plt.subplots(4,figsize = [6,12])
simsize = 30
ax[3].scatter(Sub_phup.xplott,np.array(Sub_phup.yplott)/5.5,s= simsize,color='grey')
ax[3].errorbar(np.array(Sub_phup.xplott),np.array(Sub_phup.yplott)/5.5,yerr=exterr,mfc='none',color='grey', fmt=',')
#ax[3].scatter(W1_phup.xplott,W1_phup.yplott,s= simsize,color='red',label="W1")
ax[0].scatter(J_phup.xplott,J_phup.yplott,s= simsize,color = "grey")
ax[1].scatter(H_phup.xplott,H_phup.yplott,s= simsize,color = "grey")
ax[2].scatter(K_phup.xplott,K_phup.yplott,s= simsize,color = "grey")


#for i in range(0,21,7):
colors = ['green','orange','red']
for i in range(7,13,2):
    j = int((i-7)/2)
    ax[1].errorbar(check_phup.xplott[i:i+2],check_phup.yplott[i:i+2],mfc='none',fmt='s-',color = colors[j],mec=colors[j])
    
for i in range(7,1,2):
    j = i+8
    h = int((i)/2)
    x = [check_phup.xplott[i],check_phup.xplott[j]]
    y = [check_phup.yplott[i],check_phup.yplott[j]]
    ax[1].errorbar(x,y,mfc='none',fmt='s-',color = colors[h],mec=colors[h])
ax[1].errorbar(check_phup.xplott,check_phup.yplott,mfc='none',fmt='s',color="red",mec='red')
#ax[1].errorbar(check_phup.xplott[7:14],check_phup.yplott[7:14],mfc='none',fmt='s-',color = "red",mec='red')


#ax[2].scatter(W1_phup.xplott,np.array(W1_phup.yplott)+5.5,color='red',label="W1")


for i in range(-1,2):
    ax[3].scatter(np.array(Sub_phup.xplot)[:Subcut]+i,np.array(Sub_phup.yplot[:Subcut])/5.5,s= simsize,facecolors= "none",edgecolor='black')
    #ax[3].scatter(np.array(W1_phup.xplot[:13])+i,W1_phup.yplot[:13],s= simsize,facecolors= "none",edgecolor='black')
    ax[0].scatter(np.array(J_phup.xplot[:Jcut])+i,J_phup.yplot[:Jcut],s= simsize,facecolors= "none",edgecolor='black')
    ax[1].scatter(np.array(H_phup.xplot[:Jcut])+i,H_phup.yplot[:Jcut],s= simsize,facecolors= "none",edgecolor='black')
    ax[2].scatter(np.array(K_phup.xplot[:Jcut])+i,K_phup.yplot[:Jcut],s= simsize,facecolors= "none",edgecolor='black')
    ax[3].scatter(np.array(Sub_phup.xplot[Subcut2:])+i,np.array(Sub_phup.yplot[Subcut2:])/5.5,s= simsize+5,marker="s",facecolors= "none",edgecolor='black')
    ax[0].scatter(np.array(J_phup.xplot[Jcut2:])+i,J_phup.yplot[Jcut2:],s= simsize+5,marker="s",facecolors= "none",edgecolor='black')
    ax[1].scatter(np.array(H_phup.xplot[Jcut2:])+i,H_phup.yplot[Jcut2:],s= simsize+5,marker="s",facecolors= "none",edgecolor='black')
    ax[2].scatter(np.array(K_phup.xplot[Jcut2:])+i,K_phup.yplot[Jcut2:],s= simsize+5,marker="s",facecolors= "none",edgecolor='black')
#ax[0].scatter(K_phup.xplot[:55],K_phup.data[:55],color = "red",alpha=0.2)
#ax.tick_params('x')
for eachax in ax:
    eachax.label_outer()
    eachax.grid(axis='y',color='gray',lw=0.2,alpha=0.5)
    eachax.grid(axis='x',color='gray',lw=0.2,alpha=0.5)
    eachax.set_xlim(-0.5,1.5)
    #eachax.set_xlabel(r"Phase")
    #ax.legend()
    eachax.invert_yaxis()
    #eachax.tick_params(axis='y',width=0.5)
#ax[3].legend(loc='upper right',fontsize=9)
ax[3].set_ylabel(r"m$_{850}$ [mag]")
ax[0].set_ylabel("J [mag]")
ax[1].set_ylabel("H [mag]")
ax[2].set_ylabel("K [mag]")
ax[3].set_xlabel("Phase")
plt.subplots_adjust(hspace=0)
if save:
    plt.savefig(output_dir+'PhaseD_LC_1Poffset_withcheckline.pdf')


In [71]:
(i-1)/2

3.0

In [29]:
np.array(lc.mnoises_850)

array([0.01272434, 0.01220401, 0.01393143, 0.01134586, 0.01550201,
       0.01290181, 0.01175163, 0.01172211, 0.00798045, 0.00885373,
       0.00892813, 0.00754904, 0.00662576, 0.01015805, 0.00858286,
       0.00941041, 0.00986803, 0.00895983, 0.00856453, 0.0083052 ,
       0.00932815, 0.00843354, 0.01031132, 0.01196383, 0.01045008,
       0.00950561, 0.00769993, 0.00806452, 0.00965809, 0.00758811,
       0.00751979, 0.00767007, 0.01109934, 0.0123913 , 0.00901889,
       0.00819397, 0.01005231, 0.01241434, 0.00971789, 0.00815259,
       0.0108632 , 0.01092035, 0.00822511, 0.01043903, 0.01353572,
       0.00794442, 0.01400394, 0.01036816, 0.01220165, 0.01000339,
       0.00691474, 0.00739746])

In [167]:
output_dir+'PhaseD_LC_1Poffset.pdf'

'/mnt/Secdrive/TSmain/SERPM/EC53/InCanada/New2_PhaseD_LC_1Poffset.pdf'

In [95]:
save=1
splitcolors = ["purple","orange","red","green","blue"]
marks = ['o','^','s']
simsize = 15

In [94]:
plt.close()
fig, ax = plt.subplots(2,figsize = [6,6])

ax[0].scatter(JH_phup.xplott,JH_phup.yplott,s= simsize,color = splitcolors[4])

ax[1].scatter(HK_phup.xplott,HK_phup.yplott,s= simsize,color = splitcolors[4])
for i in range(-1,2):
    for j in range(0,4):
        ax[0].scatter(np.array(JH_splitx[j])+i,JH_split[j],s= simsize,color =splitcolors[j],marker=marks[0])
        ax[1].scatter(np.array(HK_splitx[j])+i,HK_split[j],s= simsize,color =splitcolors[j],marker=marks[0])
    ax[0].scatter(np.array(JH_phup.xplot[:Jcut])+i,JH_phup.yplot[:Jcut],s= simsize+5,facecolor = "none",edgecolor='black')
    ax[1].scatter(np.array(HK_phup.xplot[:Jcut])+i,HK_phup.yplot[:Jcut],s= simsize+5,facecolor = "none",edgecolor='black')
    ax[0].scatter(np.array(JH_phup.xplot[Jcut2:])+i,JH_phup.yplot[Jcut2:],s= simsize+10,marker="s",facecolors= "none",edgecolor='black')
    ax[1].scatter(np.array(HK_phup.xplot[Jcut2:])+i,HK_phup.yplot[Jcut2:],s= simsize+10,marker="s",facecolors= "none",edgecolor='black')
    
#ax.tick_params('x')
for eachax in ax.flat:
    eachax.label_outer()
    eachax.grid(color='gray',ls='-',lw=0.2,alpha=0.5)
    eachax.set_xlim(-0.5,1.5)
    #eachax.plot([0.1,0.1],[1.5,3])
    #eachax.plot([0.3,0.3],[1.5,3])
    #eachax.plot([0.5,0.5],[1.5,3])
    #eachax.plot([0.7,0.7],[1.5,3])
    #eachax.plot([0.9,0.9],[1.5,3])
    #eachax.set_ylim(1.8,2.6)
    #eachax.set_xlabel(r"Phase")
    #ax.legend()
    #eachax.invert_yaxis()
    #plt.savefig(output_dir+'PhaseD_'+wl+'1Poffset.pdf')
ax[1].set_ylabel("H-K [mag]")
ax[0].set_ylabel("J-H [mag]")
ax[1].set_xlabel("Phase")
plt.subplots_adjust(hspace=0)
if save:
    plt.savefig(output_dir+'PhaseD_CC_1Poffset.pdf')


NameError: name 'splitcolors' is not defined

In [ ]:
len(JH_burst),len(J_burst)

plt.close()
fig, ax = plt.subplots(2, figsize= [6,10])
ax[0].scatter(JH_phup.yplot,J_phup.yplot,color = 'grey')
ax[0].set_xlabel("J-H [mag]")
ax[0].set_ylabel("J [mag]")
ax[0].scatter(JH_burst, J_burst, color= 'red',label=burstlabel)
ax[0].annotate('',xy=[2.1+0.11*3,15.3+0.265*3],xytext=[2.1,15.3],xycoords='data',arrowprops={'color':'black','arrowstyle':'->'})
ax[0].annotate('',xy=[2.1,14.5],xytext=[2.1+0.11*3,14.5+0.265*3],xycoords='data',arrowprops={'color':'black','arrowstyle':'->'})

ax[1].scatter(HK_phup.yplot,H_phup.yplot,color = 'grey')
ax[1].set_xlabel("H-K [mag]")
ax[1].set_ylabel("H [mag]")
ax[1].scatter(HK_burst, H_burst, color= 'red')
ax[1].annotate('',xy=[1.85+0.065*3,13.25+0.155*3],xytext=[1.85,13.25],xycoords='data',arrowprops={'color':'black','arrowstyle':'->'})
ax[1].annotate('',xy=[1.85,12.25],xytext=[1.85+0.065*3,12.25+0.155*3],xycoords='data',arrowprops={'color':'black','arrowstyle':'->'})

ax[0].scatter(JH_phup.yplot[:55], J_phup.yplot[:55],
              s=50,facecolor = "none",edgecolor="black",linewidths=1.3)
ax[0].legend()
ax[1].scatter(HK_phup.yplot[:55], H_phup.yplot[:55],
              s=50,facecolor = "none",edgecolor="black",linewidths=1.3)

for eachax in ax:
    eachax.invert_yaxis()
if save:
    plt.savefig(output_dir+"CMDs_JHK.pdf")

In [ ]:
print(len(JH_split[1]))
print(len(J_split[1]))

In [97]:
save=1
import matplotlib.patches as patches
from matplotlib.markers import MarkerStyle

In [99]:
plt.close()

splitlabels = []

for i in range(1,len(spliter)):
    splitlabels.append("{0:0.1f}".format(spliter[i-1]) + " < Phase < " + '{0:0.1f}'.format(spliter[i]))
fig, ax = plt.subplots(2, figsize= [6,10])
ax[0].scatter(JH_phup.yplot,J_phup.yplot,color = splitcolors[-1],
             label="{0:0.1f}".format(-0.1) + " < Phase < " + '{0:0.1f}'.format(0.1))
ax[0].set_xlabel("J-H [mag]")
ax[0].set_ylabel("J [mag]")
ax[1].scatter(HK_phup.yplot,H_phup.yplot,color = splitcolors[-1],)
for i in range(0,len(spliter)-1):
    ax[0].scatter(JH_split[i], J_split[i], color= splitcolors[i],label=splitlabels[i])
    ax[1].scatter(HK_split[i], H_split[i], color= splitcolors[i],label=splitlabels[i])

# A_v (cohen81) : J 0.265, H 0.155, K 0.090
# A_v (Rieke & Lebofsky) : J 0.282, H 0.175, K 0.112
# A_v (CCM89) : J 0.282, H 0.190, K 0.114
aJ,aH,aK = 0.282, 0.190, 0.114

ax[0].annotate('',xy=[2.12+(aJ-aH)*3,15.3+aJ*3],xytext=[2.12,15.3],xycoords='data',arrowprops={'color':'black','arrowstyle':'->'})
ax[0].annotate(r'A$_{v}$ = 3',xy=[2.12+(aJ-aH)*2.4,15.3+aJ*2.8],xycoords='data',rotation=-30)
ax[0].annotate('',xy=[2.16,14.5],xytext=[2.16+(aJ-aH)*3,14.5+aJ*3],xycoords='data',arrowprops={'color':'black','arrowstyle':'->'})
ax[0].annotate(r'A$_{v}$ = -3',xy=[2.16+(aJ-aH)*2.4,14.5+aJ*2.8],xycoords='data',rotation=-30)
#ax[0].annotate(u"\u21BA",xy=[2.3,15.25],xycoords='data',rotation = -120,size=10)

#style="Simple,tail_width=0.5,head_width=4,head_length=8"
#kw = dict(arrowstyle=style, color="k")
#a3 = patches.FancyArrowPatch((2.4,15.50), (2.4,15.55),connectionstyle="arc3,rad=5", **kw)
#ax[0].add_patch(a3)
m = MarkerStyle(r"$\circlearrowleft$")
#m = MarkerStyle(u"\u21BA")
m._transform.rotate_deg(250)

#ax[0].scatter([2.3],[15.4],marker=m,color ='grey',s=1000)
#ax[1].scatter([1.93],[12.95],marker=m,color='grey',s=1000)
ax[1].set_xlabel("H-K [mag]")
ax[1].set_ylabel("H [mag]")
ax[1].annotate('',xy=[1.8+(aH-aK)*3,13.25+aH*3],xytext=[1.8,13.25],xycoords='data',arrowprops={'color':'black','arrowstyle':'->'})
ax[1].annotate(r'A$_{v}$ = 3',xy=[1.8+(aH-aK)*2.0,13.25+aH*2.45],xycoords='data',rotation=-25)
ax[1].annotate('',xy=[1.84,12.25],xytext=[1.84+(aH-aK)*3,12.25+aH*3],xycoords='data',arrowprops={'color':'black','arrowstyle':'->'})
ax[1].annotate(r'A$_{v}$ = -3',xy=[1.84+(aH-aK)*2.6,12.25+aH*3.1],xycoords='data', rotation=-25)

ax[0].scatter(JH_phup.yplot[:Jcut], J_phup.yplot[:Jcut],
              s=50,facecolor = "none",edgecolor="black",linewidths=1.3)
ax[1].scatter(HK_phup.yplot[:Jcut], H_phup.yplot[:Jcut],
              s=50,facecolor = "none",edgecolor="black",linewidths=1.3)

ax[0].scatter(JH_phup.yplot[Jcut2:], J_phup.yplot[Jcut2:],
              s= simsize+18,marker="s",facecolors= "none",edgecolor='black')
ax[1].scatter(HK_phup.yplot[Jcut2:], H_phup.yplot[Jcut2:],
              s= simsize+18,marker="s",facecolors= "none",edgecolor='black')

for eachax in ax:
    eachax.invert_yaxis()
if save:
    plt.savefig(output_dir+"CMDs_JHK_phasesplit_pure.eps")

In [ ]:
print([len(JH_split[i]),len(J_split[i])])

In [192]:
Jcut2

156

In [85]:
aH-aK, aJ-aH

(0.076, 0.09199999999999997)

In [141]:
plt.close()

fig, ax = plt.subplots(figsize= [6,5])
ax.scatter(HK_phup.yplot,JH_phup.yplot,color = splitcolors[-1],
             label="{0:0.1f}".format(-0.1) + " < Phase < " + '{0:0.1f}'.format(0.1))
ax.set_xlabel("H-K [mag]")
ax.set_ylabel("J-H [mag]")
for i in range(0,len(spliter)-1):
    ax.scatter(HK_split[i], JH_split[i], color= splitcolors[i],label=splitlabels[i])
ax.plot(HK_phup.yplot,np.array(HK_phup.yplot)*0.58+1.12,ls='--', color= 'gray')
ax.annotate('',xy=[1.8+(aH-aK),2.3+(aJ-aH)],xytext=[1.8,2.3],xycoords='data',arrowprops={'color':'gray','arrowstyle':'->'})
ax.annotate(r'A$_{v}$ = 1',xy=[1.8,2.3+0.05],xycoords='data',color='grey',rotation=35)
#ax.legend()
ax.scatter(HK_phup.yplot[:55], JH_phup.yplot[:55],
              s=50,facecolor = "none",edgecolor="black",linewidths=1.3)
if save:
    plt.savefig(output_dir+"TCD_phasesplit.pdf")

In [ ]:
plt.close()
fig, ax = plt.subplots(1,figsize=[6,5])

ax.scatter(HK_phup.yplot,JH_phup.yplot, color='grey')
ax.scatter(HK_burst, JH_burst,color ='red',label =burstlabel)
ax.scatter(HK_phup.yplot[:55], JH_phup.yplot[:55], s=50,facecolor = "none",edgecolors="black")
ax.annotate('',xy=[1.9+0.065,2.35+0.11],xytext=[1.9,2.35],xycoords='data',arrowprops={'color':'black','arrowstyle':'->'})
ax.plot(HK_phup.yplot,np.array(HK_phup.yplot)*0.58+1.12,ls=':', color= 'gray')
ax.set_ylabel('J-H [mag]')
ax.legend(loc='upper left')
#plt.subplots_adjust(hspace=0)
if save:
    plt.savefig(output_dir+'TCD_1Poffset_burstmark.pdf')


In [ ]:
a= np.array([0,1,2,3,4,5,6])
np.select([a>3,a>5],[a,a])


In [ ]:
plt.close()
fig, ax = plt.subplots(2,figsize=[6,9])

ax[0].scatter(HK_phup.yplot,JH_phup.yplot, color='black')
ax[0].plot(HK_phup.yplot,np.array(HK_phup.yplot)*0.58+1.12,ls=':', color= 'gray')
ax[0].scatter(HK_burst, JH_burst, color='red',edgecolor = "black",label='0.4 < Phase < 0.7')
ax[0].annotate('',xy=[1.9+0.065,2.5+0.11],xytext=[1.9,2.5],xycoords='data',arrowprops={'color':'gray','arrowstyle':'->'})
ax[1].scatter(HK_phup.yplot,H_phup.yplot, color= 'black')
ax[1].scatter(HK_burst, H_burst, color='red',edgecolor = "black",label='0.4 < Phase < 0.7')
ax[1].annotate('',xy=[1.85,12.25],xytext=[1.85+0.065*3,12.25+0.155*3],xycoords='data',arrowprops={'color':'gray','arrowstyle':'->'})
ax[1].annotate('',xy=[1.85+0.065*3,13.25+0.155*3],xytext=[1.85,13.25],xycoords='data',arrowprops={'color':'gray','arrowstyle':'->'})
ax[1].invert_yaxis()
ax[1].set_xlabel('H-K [mag]')
ax[1].set_ylabel('H [mag]')
ax[0].set_ylabel('J-H [mag]')
plt.subplots_adjust(hspace=0)
ax[0].legend()
if save:
    plt.savefig(output_dir+'TCD_burstmark.pdf')

In [ ]:
#plt.close()
fig, ax = plt.subplots(2,figsize=[6,9])
J_burst = J_phup.Phasepickup([0.4,0.6])
ax[0].scatter(JH_phup.yplot, J_phup.yplot, color= 'black')
ax[0].scatter(JH_phup.yplot[:55], J_phup.yplot[:55], color= 'red',label='1st Period')
ax[0].invert_yaxis()
ax[0].set_ylabel('J [mag]')
ax[0].annotate('',xy=[2.15,14.6],xytext=[2.15+0.11*2,14.6+0.265*2],xycoords='data',arrowprops={'color':'gray','arrowstyle':'->'})
ax[0].annotate('',xy=[2.15+0.11*2,15.5+0.265*2],xytext=[2.15,15.5],xycoords='data',arrowprops={'color':'gray','arrowstyle':'->'})
ax[1].scatter(JH_phup.yplot, J_phup.yplot, color = 'black')
ax[1].scatter(JH_burst,J_burst,color= 'black', edgecolor='red', label='0.4 < Phase < 0.6')
ax[1].invert_yaxis()
ax[1].annotate('',xy=[2.15,14.6],xytext=[2.15+0.11*2,14.6+0.265*2],xycoords='data',arrowprops={'color':'gray','arrowstyle':'->'})
ax[1].annotate('',xy=[2.15+0.11*2,15.5+0.265*2],xytext=[2.15,15.5],xycoords='data',arrowprops={'color':'gray','arrowstyle':'->'})
ax[1].set_ylabel('J [mag]')
ax[1].set_xlabel('J-H [mag]')
plt.subplots_adjust(hspace=0)
ax[0].legend(loc='upper right')
ax[1].legend(loc='upper right')

In [ ]:
plt.close()


In [9]:
149597870700/696340000

214.83452149811873